In [1]:
import numpy as np
from neural_network.layer import *
from neural_network.net import *
from queue import Queue
import matplotlib.pyplot as plt
from tqdm import tqdm
from neural_network.trainutils import Dataset, train_test_split
import os

In [2]:
current_dir = os.getcwd()
face_dir = os.path.join(current_dir, "cmu+face+images", "faces_4")
dataset = Dataset(face_dir)

In [3]:
dataset.data.shape

(624, 30, 32)

In [4]:
train, validation, test = train_test_split(dataset, (0.8, 0.0, 0.2))

In [5]:
dataset[500]

(array([ 51,  49,  45,  43,  39,  37,  35,  31,  29,  24,  20,  18,  24,
         24,  27,  18,  12,  18,  22,  27,  29,  31,  33,  37,  39,  43,
         45,  47,  49,  49,  49,  51,  51,  49,  45,  43,  41,  37,  35,
         33,  29,  27,  24,  27,  31,  24,  31,  24,  18,  24,  29,  31,
         33,  35,  39,  41,  43,  45,  47,  47,  47,  49,  49,  51,  51,
         49,  45,  45,  43,  39,  37,  35,  33,  31,  29,  33,  37,  27,
         33,  31,  24,  31,  35,  37,  39,  39,  41,  43,  45,  45,  47,
         49,  49,  51,  51,  51,  51,  49,  47,  45,  43,  41,  39,  37,
         35,  33,  31,  35,  41,  29,  39,  35,  31,  37,  39,  41,  41,
         43,  45,  45,  47,  49,  51,  51,  51,  51,  51,  51,  51,  49,
         49,  47,  45,  43,  41,  39,  37,  37,  35,  37,  43,  33,  41,
         39,  35,  39,  43,  43,  45,  45,  47,  49,  51,  51,  51,  51,
         51,  51,  51,  51,  49,  49,  49,  47,  45,  45,  43,  41,  39,
         37,  35,  39,  45,  35,  43,  41,  37,  41

In [6]:
input_shape = train[0][0].shape[0]
input_shape

960

In [7]:
train[0]

(array([ 4.17444925e-01,  5.08948977e-02, -5.67590739e-01, -6.01203465e-01,
        -6.21927062e-01, -7.97935912e-01, -1.09963865e+00, -1.23134098e+00,
        -1.14150151e+00, -1.95398850e+00, -1.77580437e+00, -1.65122426e+00,
        -1.64326483e+00, -1.91264776e+00, -2.02503188e+00, -1.90132753e+00,
        -1.77277759e+00, -1.87897455e+00, -1.60376668e+00, -1.52262927e+00,
        -1.18069969e+00, -8.41236779e-01, -7.67324143e-01, -5.34905379e-01,
        -6.59481722e-01, -4.02765332e-01, -4.06312150e-02,  1.16518056e-01,
         1.37777183e-01,  2.98655995e-01,  3.79791933e-01,  3.77812090e-01,
         4.80505406e-01,  1.21045240e-01, -7.45525454e-02, -2.04407898e-01,
        -7.42815975e-01, -8.30200595e-01, -8.14730505e-01, -8.68110801e-01,
        -9.50059585e-01, -1.55150000e+00, -1.33715198e+00, -1.17062797e+00,
        -1.07915722e+00, -1.34671155e+00, -1.90307178e+00, -1.58503922e+00,
        -1.20298307e+00, -8.55995078e-01, -4.48954340e-01, -1.92434460e-01,
         3.0

In [8]:
linear1 = Linear(input_shape, input_shape//2)
relu1 = ReLU(linear1)
linear2 = Linear(input_shape//2, input_shape//4, relu1)
relu2 = ReLU(linear2)
linear3 = Linear(input_shape//4, input_shape//8, relu2)
relu3 = ReLU(linear3)
linear4 = Linear(input_shape//8, input_shape//16, relu3)
relu4 = ReLU(linear4)
linear5 = Linear(input_shape//16, len(dataset.keys), relu4)
softmaxlayer = Softmax(linear5)
loss_layer = CrossEntropy(softmaxlayer)

model = NeuralNetwork(linear1, softmaxlayer, loss_layer)

In [9]:
print(model.n_layers)
print(model.output_layer)

print("Forward order:")
for layer in model._forward_order:
    print(layer)

print("\nBackward order:")
for layer in model._backward_order:
    print(layer)

11
Forward order:
Linear: (960, 1) -> (480, 1)
ReLU
Linear: (480, 1) -> (240, 1)
ReLU
Linear: (240, 1) -> (120, 1)
ReLU
Linear: (120, 1) -> (60, 1)
ReLU
Linear: (60, 1) -> (4, 1)
Softmax
CrossEntropy

Backward order:
CrossEntropy
Softmax
Linear: (60, 1) -> (4, 1)
ReLU
Linear: (120, 1) -> (60, 1)
ReLU
Linear: (240, 1) -> (120, 1)
ReLU
Linear: (480, 1) -> (240, 1)
ReLU
Linear: (960, 1) -> (480, 1)


In [13]:
loss_array = []
pbar = tqdm(range(100))
for i in pbar:
    epoch_loss = 0
    for data, label in train:
        data = data.reshape(-1, 1)
        out = model.forward(data)
        #print(out)
        loss = model.loss_layer.forward(out, label)
        epoch_loss += loss
        model.backward()
        model.step()
    epoch_loss = epoch_loss / len(train)
    loss_array.append(epoch_loss)
    pbar.set_description(float(epoch_loss))

  0%|          | 0/100 [00:00<?, ?it/s]/tmp/ipykernel_3587708/2986568807.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pbar.set_description(float(epoch_loss))
  0%|          | 0/100 [00:01<?, ?it/s]


TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [ ]:
linear5.out

array([[ 0.00264682],
       [-0.00232187],
       [ 0.00670767],
       [ 0.00604786]])

In [ ]:
np.exp(linear1.out)

array([[0.64656248],
       [1.4592194 ],
       [0.6687198 ],
       [1.363603  ],
       [1.00368401],
       [1.05412504],
       [0.75358436],
       [1.3410314 ],
       [1.31333321],
       [0.82018137],
       [1.23795864],
       [1.01179897],
       [0.8704236 ],
       [0.83459677],
       [1.07165763],
       [1.35874341],
       [0.53754426],
       [1.0159353 ],
       [0.74244091],
       [0.99880028],
       [1.10227364],
       [1.02366985],
       [1.07736906],
       [0.75661777],
       [1.02166014],
       [1.00230895],
       [0.98085862],
       [0.50825743],
       [0.96272453],
       [0.76743918],
       [1.09709379],
       [1.14694883],
       [1.02610712],
       [1.41528296],
       [1.17164127],
       [1.64165865],
       [1.01908742],
       [0.87527966],
       [0.76734304],
       [0.89691066],
       [1.00726944],
       [0.82749108],
       [1.60457411],
       [1.20254267],
       [0.71926218],
       [1.18046516],
       [1.40149603],
       [0.997

In [ ]:
dataset.data.shape

(624, 30, 32)

In [ ]:
print(len(train))

499
